In [1]:
import pandas as pd
path="melb_data.csv"

# Reading CSV file

In [2]:
data=pd.read_csv(path)

In [3]:
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


## selecting the target

In [4]:
y=data.Price
y.head()

0    1480000.0
1    1035000.0
2    1465000.0
3     850000.0
4    1600000.0
Name: Price, dtype: float64

### to make simple we excluded object types from data

In [5]:
melb_predectors=data.drop(['Price'], axis=1)
X=melb_predectors.select_dtypes(exclude=['object'])
X.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


### dividing the data in train and validate

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train , y_valid=train_test_split(X,y,train_size=0.8, test_size=0.2)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


In [8]:
def select_mae(X_train, X_valid, y_train, y_valid):
    model=RandomForestRegressor()
    model.fit(X_train,y_train)
    preds=model.predict(X_valid)
    return mean_absolute_error(y_valid,preds)

#### using droping column techniques

In [9]:
cols_missing_data=[
    column for column in X_train.columns
    if X_train[column].isnull().any()
]

#### dropping the missing datacolumns 

In [10]:
new_X_train =X_train.drop(cols_missing_data, axis=1)
new_X_valid =X_valid.drop(cols_missing_data, axis=1)

In [11]:
print("MAE from Dropping Table is:",select_mae(new_X_train, new_X_valid, y_train, y_valid))

MAE from Dropping Table is: 180540.402990918


## from approach 2 imputation

In [12]:
from sklearn.impute import SimpleImputer
Imputer=SimpleImputer()

In [13]:
imputed_X_train=pd.DataFrame(Imputer.fit_transform(X_train))

In [14]:
imputed_X_valid=pd.DataFrame(Imputer.fit_transform(X_valid))
imputed_X_train.columns=X_train.columns
imputed_X_valid.columns=X_valid.columns

In [15]:
print("MAE from Dropping Table is:",select_mae(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Dropping Table is: 181325.42778613506


### approach 3 using extension imputation

In [16]:
X_train_plus=X_train.copy()
X_valid_plus=X_valid.copy()

##### making new column by indicating the imputed column

In [17]:
cols_missing_data

['Car', 'BuildingArea', 'YearBuilt']

In [24]:
for col in cols_missing_data:
     X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
     X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
    


In [25]:
 X_train_plus

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
4088,2,6.9,3039.0,2.0,1.0,1.0,0.0,39.0,2014.0,-37.77150,144.91600,6232.0,False,False,False
2792,3,9.2,3146.0,3.0,2.0,3.0,607.0,NaN,NaN,-37.85340,145.07350,10412.0,False,True,True
10248,2,7.2,3184.0,2.0,1.0,0.0,702.0,86.0,1940.0,-37.87944,144.99375,8989.0,False,False,False
6028,4,13.3,3020.0,4.0,1.0,1.0,609.0,NaN,NaN,-37.76610,144.83960,4217.0,False,True,True
579,2,9.7,3103.0,2.0,1.0,1.0,130.0,80.0,1960.0,-37.81200,145.08760,5682.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11831,3,15.5,3038.0,3.0,2.0,1.0,655.0,NaN,NaN,-37.70894,144.80205,5336.0,False,True,True
2276,2,7.7,3184.0,2.0,1.0,2.0,0.0,86.0,1930.0,-37.87400,144.98190,8989.0,False,False,False
5547,2,2.1,3205.0,2.0,1.0,1.0,0.0,NaN,NaN,-37.83460,144.96730,5943.0,False,True,True
7968,2,7.5,3123.0,2.0,1.0,2.0,0.0,64.0,1970.0,-37.82540,145.04740,6482.0,False,False,False


In [19]:
myimputer=SimpleImputer()
imputed_x_train_plus=pd.DataFrame(myimputer.fit_transform(X_train_plus))
imputed_x_valid_plus=pd.DataFrame(myimputer.fit_transform(X_valid_plus))
imputed_x_train_plus.columns=X_train_plus.columns
imputed_x_valid_plus.columns=X_valid_plus.columns

In [21]:
print("MAE from extension imputation:",select_mae(imputed_x_train_plus, imputed_x_valid_plus, y_train, y_valid))

MAE from extension imputation: 179297.07037493863
